https://www.sociomix.com/diaries/entertainment/the-problems-with-cancel-culture-and-popular-youtubers/1627615253

https://www.speakrj.com/audit/report/UCX6OQ3DkcsbYNE6H8uQQuVA/youtube

https://developers.google.com/youtube/v3

https://socialblade.com/youtube/user/shane